# Folk $n$-gram Analysis (FONN)

*FONN* (pronounced "fun") is an Irish (*Gaeilge*) word for "tune".

In this corpus we present two Polifonia components:

1. aeflkjaef
2. aeflkeajflakjf

This Demo notebook will demonstrate how we use them so far.

### Prerequisites

* In `<basepath>/MIDI` we should have a corpus of folk tunes in MIDI format. By default `basepath` is `./corpus/`. If the corpus is elsewhere, change `basepath` below. We will be writing outputs to subdirectories of `basepath`.

* Install the libraries *Feather*, *PyArrow*, *fastDamerauLevenshtein*, and *Music21*:

    `pip install feather music21 pyarrow fastDamerauLevenshtein`
    
### TODO

* Some TODO notes throughout
* Maybe we should check whether the primary/secondary sequences and n-grams files exist, before running for 40m

In [12]:
import os.path
import sys
sys.path.append("setup_corpus") # TODO we should be able to remove this by making setup_corpus a proper module

import pandas as pd
pd.options.mode.chained_assignment = None
from fastDamerauLevenshtein import damerauLevenshtein

In [13]:
basepath = "./corpus/"
inpath = basepath + "/MIDI"
roots_path = basepath + "/roots.csv"
feat_seq_path=basepath + "/feat_seq_data/note"
accents_path=basepath + "/feat_seq_data/accent"
duration_weighted_path=basepath + "/feat_seq_data/duration_weighted"
ngram_inpath = basepath + "/feat_seq_data/accent"
ngram_outpath = basepath + "/ngrams"
ngram_sim_inpath = basepath + "/ngrams/cre_pitch_class_accents_ngrams_tfidf.ftr" # please check
for path in [inpath, feat_seq_path, accents_path, duration_weighted_path, 
             ngram_inpath, ngram_outpath, ngram_sim_inpath]:
    assert os.path.exists(path)

These two Python scripts contain tools for reading the MIDI data, processing it to find the primary and secondary feature sequences, key-invariant sequences, and duration-weighted sequences.

**TODO** define "primary and secondary feature sequences" very briefly in the notebook (already described in other deliverable of course).

In [16]:
import setup_corpus.setup_corpus as setup_corpus
from setup_corpus.corpus_processing_tools import Music21Corpus, MusicDataCorpus


Setting up lookup table for root assignment:
  note names  midi num  root num
0          C        60         0
1   C# or D-        61         1
2          D        62         2
3   D# or E-        63         3
4          E        64         4 



Setting up Music21 root detection lookup table:
  note name  pitch class
0         C            0
1        C#            1
2        D-            1
3         D            2
4        D#            3 




In [18]:
m21_corpus = Music21Corpus(inpath)

Input corpus contains 1224 melodies:

Tureengarbh Jig, The
Young And Stylish
Fun at the Fair
Buckley the Fiddler  (reel)
Tommy Coen's Reel
Lynch's Hornpipe
Tuttle's Reel
Patrick O'Connor's  (polka)
Patsy Tuohey's (reel)
Beggarman's Reel, The
Up and About In the Morning  (slide)
My Love Between Two Roses (reel)
Ladies Step Up to Tea!
Old Pigeon on the Gate, The
Curragh Races (reel), The
Mangan's Fancy (reel)
McDonagh's Reel 1
One of Tommy's  (hornpipe)
Listowel Fiddler  (slide), The
John Flynn's Jig
Callan Lasses (reel), The
Knocknaboul Reel, The
Jackson's Post-Chaise
Father Skehan's Jig
Tom Connors' Jig
Sailor's Bonnet (reel), The
Pomeroy Fiddler  (reel), The
Her Golden Hair Was Curling Down
Frog in the Puddle , The
Tap the Barrel (reel)
Trip to Durrow (reel), The
Humors of Derrycros(s)ane, The
Nine Mile House  (reel)
Duke of Leinster Hornpipe, The
Mason's Apron (reel), The
Pat O'Beirne's Favorite
Gooseberry Bush (reel), The
Carty's Reel
Brother Gildas' Jig
Hunter's Purse (reel), The
L

Running the following cell will take about 15 minutes. It will produce many `csv` files under `<basepath>/feat_seq_data/note`, `<basepath>/feat_seq_data/accent`, `<basepath>/feat_seq_data/duration_weighted`. 

In [ ]:
corpus = setup_corpus.SetupCorpus(m21_corpus)
corpus.generate_primary_feat_seqs()
corpus.setup_music_data_corpus()
corpus.run_simple_secondary_feature_sequence_calculations()
corpus.run_key_invariant_sequence_calulations(roots_path)
corpus.run_duration_weighted_sequence_calculations(['pitch', 'pitch_class'])
corpus.save_corpus(
    feat_seq_path=feat_seq_path,
    accents_path=accents_path,
    duration_weighted_path=duration_weighted_path
)



For example, in `A Trip To Galway_note.csv`, the first few lines will be:

In [6]:
df = pd.read_csv(basepath + "/feat_seq_data/note/A Trip To Galway_note.csv")
df.head()

,Unnamed: 0,MIDI_note,onset,duration,velocity,interval,parsons_code,Parsons_cumsum,chromatic_root,pitch,pitch_class
0,0,74,0.0,1.0,105,0,0,0,4,10,10
1,1,71,1.0,1.0,105,-3,-1,-1,4,7,7
2,2,67,2.0,1.0,80,-4,-1,-2,4,3,3
3,3,64,3.0,1.0,80,-3,-1,-3,4,0,0
4,4,64,4.0,1.0,95,0,0,-3,4,0,0


Here, we see that **TODO** add here

Next, we will calculate the most important $n$-grams in each tune, calculating importance using TF-IDF.

In [7]:
from ngram_tfidf_tools import NgramCorpus
from setup_ngrams_tfidf import SetupNgramsTfidf

Again, the following cell will take about 25 minutes to run.

In [8]:
feature = "pitch_class"
n_vals = list(range(5, 10))
feat_seq_corpus = NgramCorpus(ngram_inpath)
ngram_corpus = SetupNgramsTfidf(feat_seq_corpus, feature, n_vals)
ngram_corpus.extract_ngrams()
ngram_corpus.calculate_tfidf()
ngram_corpus.save_results(outpath=ngram_outpath,
                          corpus_name='cre_pitch_class_accents')


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Primrose Girl (reel), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          62    0.0      1.00       105         0             0   
4          72    4.0      1.00        95        10             1   
8          71    8.0      1.00       105        -1            -1   
12         69   12.0      0.66        95        -2            -1   
17         62   16.0      1.00       105        -7            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               7     -5            7  
4                1               7      5            5  
8                0               7      4            4  
12              -1               7      2            2  
17              -2               7     -5            7  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Irishman's Blackthorn (reel), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          76    0.0       2.0       105         0             0   
3          72    4.0       2.0        95        -4            -1   
5          74    8.0       1.0       105         2             1   
9          74   12.0       1.0        95         0             0   
14         76   16.0       2.0       105         2             1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               9      7            7  
3               -1               9      3            3  
5                0               9      5            5  
9                0               9      5            5  
14               1               9      7            7  

Reading data from:
/Users/jmmcd/OneDrive - National University of


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Brian o Laimhin (reel)_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          67    0.0       2.0       105         0             0   
3          79    4.0       1.0        95        12             1   
7          67    8.0       1.0       105       -12            -1   
11         72   12.0       1.0        95         5             1   
15         67   16.0       2.0       105        -5            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               7      0            0  
3                1               7     12            0  
7                0               7      0            0  
11               1               7      5            5  
15               0               7      0            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, G


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Bill Harte's_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          62    0.0       1.0       105         0             0   
3          69    3.0       2.0        95         7             1   
5          71    6.0       1.0       105         2             1   
8          69    9.0       1.0        95        -2            -1   
11         62   12.0       1.0       105        -7            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               2      0            0  
3                1               2      7            7  
5                2               2      9            9  
8                1               2      7            7  
11               0               2      0            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitH

    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          79    0.0       1.0       105         0             0   
2          76    2.0       1.0       105        -3            -1   
7          71    6.0       1.0        95        -5            -1   
11         74   10.0       1.0       105         3             1   
15         69   14.0       1.0        95        -5            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               4     15            3  
2               -1               4     12            0  
7               -2               4      7            7  
11              -1               4     10           10  
15              -2               4      5            5  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Jimmy Ward's Jig_accent.csv
   MIDI_note  onset  duration  velocity  interval  parsons_code  \
0         62    

...


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Lovely Lassie Winking (reel)_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          66    0.0       1.0       105         0             0   
4          71    4.0       1.0        95         5             1   
8          71    8.0       1.0       105         0             0   
12         71   12.0       1.0        95         0             0   
15         66   16.0       1.0       105        -5            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               2      4            4  
4                1               2      9            9  
8                1               2      9            9  
12               1               2      9            9  
15               0               2      4            4  

Reading data from:
/Users/jmmcd/OneDrive - National University of Irel


Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Girls of the County Mayo (reel), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          76    0.0       1.0       105         0             0   
3          76    4.0       1.0        95         0             0   
7          69    8.0       2.0       105        -7            -1   
10         71   12.0       1.0        95         2             1   
13         76   16.0       1.0       105         5             1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               7      9            9  
3                0               7      9            9  
7               -1               7      2            2  
10               0               7      4            4  
13               1               7      9            9  

Reading data from:
/Users/jmmcd/OneDrive - National University

16               0               4      7            7  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Flowers of Spring, The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          67    0.0       1.0       105         0             0   
1          69    1.0       1.0       105         2             1   
4          69    4.0       1.0        95         0             0   
7          76    7.0       1.0       105         7             1   
10         67   10.0       2.0        95        -9            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               9     -2           10  
1                1               9      0            0  
4                1               9      0            0  
7                2               9      7            7  
10               1               9     -2           10  

Reading data from:


14               0               7      0            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Tap the Barrel (reel)_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          62    0.0       2.0       105         0             0   
4          69    4.0       1.0        95         7             1   
9          72    8.0       1.0       105         3             1   
13         60   12.0       1.0        95       -12            -1   
17         62   16.0       2.0       105         2             1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               2      0            0  
4                1               2      7            7  
9                2               2     10           10  
13               1               2     -2           10  
17               2               2      0            0  

Reading data from:
/

13               1               2     12            0  

Reading data from:
/Users/jmmcd/OneDrive - National University of Ireland, Galway/GitHub/folk_ngram_analysis/corpus/feat_seq_data/accent/Master's Sporting Paddy (reel), The_accent.csv
    MIDI_note  onset  duration  velocity  interval  parsons_code  \
0          67    0.0       1.0       105         0             0   
2          71    2.0       2.0       105         4             1   
5          62    6.0       1.0        95        -9            -1   
9          71   10.0       2.0       105         9             1   
12         67   14.0       1.0        95        -4            -1   

    Parsons_cumsum  chromatic_root  pitch  pitch_class  
0                0               7      0            0  
2                1               7      4            4  
5                0               7     -5            7  
9                1               7      4            4  
12               0               7      0            0  

Readin

Now we have TF-IDF scores for each $n$-gram for each tune. For example:

In [9]:
df = pd.read_csv(basepath + "/ngrams/cre_pitch_class_accents_ngrams_freq.csv")
df.head()


,Unnamed: 0,ngram,"Primrose Girl (reel), The_accent_pitch_class_freq","Soldier's Joy, The_accent_pitch_class_freq","Sligo Jig, The_accent_pitch_class_freq","Rambling Connachtman (reel), The_accent_pitch_class_freq",Orange and Green_accent_pitch_class_freq,"London Lasses (reel), The_accent_pitch_class_freq",Dusty Miller_accent_pitch_class_freq,"Rainy Day Jig, The_accent_pitch_class_freq",...,Tear the Calico (reel)_accent_pitch_class_freq,Jack's Alive (reel)_accent_pitch_class_freq,"Tirnaskea Lasses (reel), The_accent_pitch_class_freq",Roaring Mary (reel)_accent_pitch_class_freq,"Master's Sporting Paddy (reel), The_accent_pitch_class_freq","Chattering Magpie (reel), The_accent_pitch_class_freq",Scully Casey's ( ) (hornpipe)_accent_pitch_class_freq,"Ranger (h'pipe), The_accent_pitch_class_freq",freq,idf
0,0,"(0, 0, 0, 0, 0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,595,6.225
1,1,"(0, 0, 0, 0, 0, 0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,494,6.410
2,2,"(0, 0, 0, 0, 0, 0, 0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,456,6.490
3,3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,435,6.537
4,4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,425,6.561


**TODO** Explain a little of the above, or else print out something else to show what we have got from the ngrams.

In [10]:
df[["ngram", "A Trip To Galway_accent_pitch_class_freq", "freq", "idf"]]

,ngram,A Trip To Galway_accent_pitch_class_freq,freq,idf
0,"(0, 0, 0, 0, 0)",0,595,6.225
1,"(0, 0, 0, 0, 0, 0)",0,494,6.410
2,"(0, 0, 0, 0, 0, 0, 0)",0,456,6.490
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0,435,6.537
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0,425,6.561
...,...,...,...,...
495,"(7, 7, 0, 0, 4)",0,19,9.667
496,"(10, 10, 0, 2, 10)",0,19,9.667
497,"(7, 10, 7, 10, 7, 10)",0,19,9.667
498,"(0, 5, 0, 0, 4)",0,19,9.667


Finally, we will demonstrate some work-in-progress for calculating similarity between tunes, based on similarity between between their $n$-grams. This uses the Damerau-Levenshtein algorithm.

In [11]:
from ngram_pattern_search import NgramSimilarity

In [12]:
pattern_search = NgramSimilarity(ngram_sim_inpath)
pattern_search.extract_candidate_ngrams("Lord McDonald's (reel)", n=6, mode='idx', indices=[0, 1])
pattern_search.setup_test_corpus()
pattern_search.find_similar_patterns(edit_dist_threshold=1)
pattern_search.find_similar_tunes()
print(pattern_search.results)

                            ngram  \
0  (2.0, 7.0, 2.0, 7.0, 4.0, 7.0)   
1  (4.0, 4.0, 7.0, 7.0, 2.0, 4.0)   
2  (7.0, 2.0, 4.0, 0.0, 7.0, 4.0)   
3  (7.0, 4.0, 7.0, 2.0, 7.0, 2.0)   
4  (7.0, 2.0, 4.0, 7.0, 2.0, 4.0)   

   Primrose Girl (reel), The_accent_pitch_class_tfidf  \
0                                                0.0    
1                                                0.0    
2                                                0.0    
3                                                0.0    
4                                                0.0    

   Soldier's Joy, The_accent_pitch_class_tfidf  \
0                                          0.0   
1                                          0.0   
2                                          0.0   
3                                          0.0   
4                                          0.0   

   Sligo Jig, The_accent_pitch_class_tfidf  \
0                                      0.0   
1                                      0.

Corpus n-gram filtering complete.

Searching corpus for similar n-gram patterns...
57 Similar patterns detected:
                            ngram  (2.0, 7.0, 2.0, 7.0, 4.0, 7.0)  \
0  (2.0, 7.0, 2.0, 7.0, 2.0, 7.0)                             1.0   
1       (4.0, 4.0, 7.0, 7.0, 4.0)                             4.0   
2       (7.0, 2.0, 7.0, 4.0, 7.0)                             1.0   
3  (2.0, 7.0, 4.0, 7.0, 4.0, 7.0)                             1.0   
4       (2.0, 7.0, 2.0, 7.0, 4.0)                             1.0   

   (4.0, 4.0, 7.0, 7.0, 2.0, 4.0)  
0                             4.0  
1                             1.0  
2                             4.0  
3                             4.0  
4                             3.0  
Searching corpus for similar tunes...
Similarity results for Lord McDonald's (reel):
                                           title  count
33      Lord McDonald's (reel)_accent_pitch_clas     15
41       Tim Mulloney's (reel)_accent_pitch_clas      5
53 

**TODO** explain something about the above results, or show some more interesting results.